<a href="https://colab.research.google.com/github/Winzen/mides-rascunho/blob/main/code/scraping/to/%5Bto%5Dextrair.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Constantes

In [ ]:
ano = 2018
csv_entidade_git = "https://raw.githubusercontent.com/Winzen/mides-rascunho/main/code/scraping/to/municipios-entidades/entidades_to.csv?token=GHSAT0AAAAAACJFESHXRUQPOWT6TB5XH3QSZTQ44KQ"
path_drive_input = None # Definir caso você queira salvar os arquivos em algum lugar automaticamente ou extrair eles


# Verificar IP

In [ ]:
import requests

my_country = requests.get("https://api.myip.com/")
my_country.json()

{'ip': '34.150.148.240', 'country': 'United States', 'cc': 'US'}

# Importação

In [ ]:
from typing import IO
import unicodedata
import re
from urllib3 import PoolManager, disable_warnings, exceptions, Timeout, Retry
import urllib.request
from bs4 import BeautifulSoup
import concurrent.futures
import pandas as pd
from more_itertools import batched
import os
import time
from time import sleep
import shutil
from zipfile import ZipFile
import gc
import glob
import requests


class ReponseManage:

  def __init__(self, pool_max_size: int = 1000) -> None:

      http = self.create_pool_manager(pool_max_size)

  def create_pool_manager(self, pool_max_size: int) -> PoolManager:

      http = PoolManager(maxsize=pool_max_size, cert_reqs='CERT_NONE', block=True)
      disable_warnings(exceptions.InsecureRequestWarning)

      return http

class ColetaEntidade:

    def __init__(self, ano: str, id_municipio: str,
                 id_entidade: str, categoria: str, link: str) -> None:

      self.ano = ano
      self.id_municipio = id_municipio
      self.id_entidade = id_entidade
      self.categoria = categoria
      self.link = link


def slugify(s: str):
        s = s.strip().lower()
        s = unicodedata.normalize("NFD", s)
        s = s.encode("ascii", "ignore")
        s = s.decode("utf-8")
        s = s.lower().strip()
        s = re.sub(r'[^\w\s-]', '', s)
        s = re.sub(r'[\s_-]+', '_', s)
        s = re.sub(r'^-+|-+$', '', s)
        return s



def form_link_entidade(pool_manager: PoolManager, id_municipio: int | str, ano: int | str) -> list:

  base_url = "https://portaldocidadao.tce.to.gov.br/estadomunicipios/getUnidades/"

  fields = {
        "municipio": id_municipio,
        "exercicio": ano,
        "remessa": "8",
        "ajax": "true"
    }

  entidades = http.request('GET', base_url, fields=fields).json()

  for entidade in entidades:
    entidade["id_municipio"] = id_municipio
    entidade["ano"] = ano

  return entidades



def form_link_to_data(row: pd.DataFrame,
                      categoria: str = "empenho") -> ColetaEntidade:
  url = f"https://www.tce.to.gov.br/sicap/contabilx/relatorios/planilhas/"
  parametros = f"{categoria}.php?unidade={row.id}&exercicio={row.ano}&bimestre=6&status=1&conferencia=1"
  link = url + parametros
  slot = ColetaEntidade(row.ano, row.id_municipio, row.id, categoria, link)
  return slot


def extrair_entidade(http: PoolManager, row: ColetaEntidade) -> None:

  table = get_table(http, row.link)
  table_save(row, table)

def table_save(row: ColetaEntidade, html: IO) -> None:

  dado = pd.read_html(html, header=0)
  dado = dado[0]
  dado.columns = dado.columns.str.replace(". ", "_").str.lower()
  dado.columns = [slugify(column) for column in dado.columns]

  file_name = f"{row.categoria}.csv"
  base_path = os.path.join(os.getcwd(), "input", row.ano,
                           row.id_municipio,
                           row.id_entidade)

  path_file = os.path.join(base_path, file_name)

  os.makedirs(base_path, exist_ok=True)

  dado.to_csv(path_file, index=False)

  del dado

def download_dado(row: ColetaEntidade):

  file_name = f"{row.categoria}.xls"
  base_path = os.path.join(os.getcwd(), "input", row.ano,
                          row.id_municipio,
                          row.id_entidade)

  path_file = os.path.join(base_path, file_name)

  os.makedirs(base_path, exist_ok=True)
  chunk_size = 1000000 # Define the chunk size (in bytes)

  # Open the URL
  r = requests.get(row.link, stream=True)

  with open(path_file, 'wb') as fd:
      fd.write(r.content)
      for chunk in r.iter_content(chunk_size=chunk_size):
          fd.write(chunk)

def get_table(http: PoolManager, link: str) -> bytes:

    table = http.request('GET', link).data
    return table


def verify_exists(row: pd.DataFrame) -> bool:

    file_name = f"{row.categoria}.xls"
    base_path = os.path.join(os.getcwd(), "input", row.ano,
                            row.id_municipio,
                            row.id_entidade)

    path_file = os.path.join(base_path, file_name)

    path_csv_antigo = str(path_file).replace(".xls", ".csv")

    if not os.path.exists(path_file) and not os.path.exists(path_csv_antigo):
        return True

    return False



def send_folder_drive(path_drive: str, path_input: str) -> None:

  if not len(os.listdir(path_input)) > 0 or not path_drive:
    raise Exception("Essa Pasta vazia")
    return None

  shutil.make_archive(path_drive,
                      'zip',
                      path_input)


def send_solo_drive() -> None:

  if path_drive_input:
    drive.mount('/content/drive')

    send_folder_drive(path_drive_input + "_auto",
                      "/content/input/")
  else:

    print("Caminho para auto-save não definido")

def extrair(path, path_output) -> None:
  # loading the temp.zip and creating a zip object
  with ZipFile(path, 'r') as zObject:

      # Extracting all the members of the zip
      # into a specific location.
      zObject.extractall(
          path=path_output)


# Extrair

In [ ]:
extrair(path=path_drive_input + '.zip',
        path_output="/content/input")

### Extrair Auto-Save

In [ ]:
extrair(path=path_drive_input + "_auto" + '.zip',
        path_output="/content/input")

# Registrar ID municipios e Entidades

In [ ]:
http = PoolManager(maxsize=139, cert_reqs='CERT_NONE', block=True)
disable_warnings(exceptions.InsecureRequestWarning)

link = "https://portaldocidadao.tce.to.gov.br/estadomunicipios/index"
response = http.request('GET', link)

txt_html = response.data
soup = BeautifulSoup(txt_html, 'html.parser')

elementos_municipios = soup.select('select[name="comboMunicipio"] option')[1:]
anos = soup.select('select[name="comboExercicio"] option')[1:]
anos = [ano.get("value") for ano in anos]

dataframe = {elemento.text: elemento.get("value") for elemento in elementos_municipios}

entidades = []
for ano in anos:
    with concurrent.futures.ThreadPoolExecutor() as executor:
      entidades += [entidades_json
                    for entidades_json in
                    executor.map(
                    lambda id_municipio: form_link_entidade(http, id_municipio, ano),
                    dataframe.values()
                    )]


entidades_to_df = [entidade for entidades_list in entidades for entidade in entidades_list]
df_entindade = pd.DataFrame(entidades_to_df, dtype=str)
df_entindade.columns = df_entindade.columns.str.lower()
df_entindade.to_csv("entidades_to.csv", index=False)

# Raspagem

In [ ]:
entidades = pd.read_csv(csv_entidade_git, dtype=str)

In [ ]:
chucksize = 30

entidade_ano_coleta = entidades[entidades.ano == str(ano)]
categorias = ["empenho", "liquidacao", "pagamento"]
linha_coleta = [form_link_to_data(entidade, categoria)
                for entidade in entidade_ano_coleta.itertuples()
                for categoria in categorias]

chunks = list(batched(linha_coleta, chucksize))

for n, chunk in enumerate(chunks):

  try:

    start_time = time.time()
    print(f"Chunk: {n+1}/{len(chunks)} Iniciado\n")

    rows_to_extract = [row for row in chunk if verify_exists(row)]

    if rows_to_extract:

      with concurrent.futures.ThreadPoolExecutor() as executor:
        executor.map(download_dado, rows_to_extract)

      if (n + 1) % 2 == 0:
        send_solo_drive()

      sleep(1)

    print(f"Duração da execução: {(time.time() - start_time) / 60:.2f}\n{'-' * 30}")


  except Exception as erro:
    print(f"Erro: {erro}\nFalha chunk {n+1}")
    sleep(10)


send_solo_drive()


# Verificar dados perdidos

In [ ]:
path = f"/content/input/{ano}/**/**/*"
files = glob.glob(path)

ids_entidades_coletados = [path.split("/")[-2] for path in files]
ids_entidades_coletados = list(dict.fromkeys(ids_entidades_coletados))

mask = ~entidade_ano_coleta.id.isin(ids_entidades_coletados)
nao_coletados = entidade_ano_coleta[mask]

categorias = ["empenho", "liquidacao", "pagamento"]
nao_coletados = [form_link_to_data(entidade, categoria)
                for entidade in nao_coletados.itertuples()
                for categoria in categorias]

for row in nao_coletados:
  print(row.link)

# Mandar para o drive Manualmente

In [ ]:
drive.mount('/content/drive')


send_folder_drive(path_drive_input,
                f"/content/input/")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
